### 作業目的: 熟練Torchtext進行文本資料讀取

本次作業主要會使用[polarity](http://www.cs.cornell.edu/people/pabo/movie-review-data/)的電影評論來進行torchtext資料讀取，學員可以在附件的polarity.tsv看到所使用的資料。

Hint: 這次作業同學可以嘗試使用[torchtext.data.TabularDataset](https://torchtext.readthedocs.io/en/latest/data.html#tabulardataset)，更簡易讀取資料

### 載入套件

In [1]:
import torch
import numpy as np
import pandas as pd
from torchtext import data, datasets

In [2]:
# 探索資料
# 可以發現資料為文本與類別，而類別即為正評與負評
input_data = pd.read_csv('polarity.tsv', delimiter='\t', header=None, names=['text', 'label'])
input_data

,text,label
0,films adapted from comic books have had plenty...,1
1,every now and then a movie comes along from a ...,1
2,you've got mail works alot better than it dese...,1
3,jaws is a rare film that grabs your attentio...,1
4,moviemaking is a lot like being the general ma...,1
...,...,...
1995,"if anything , "" stigmata "" should be taken as ...",0
1996,"john boorman's "" zardoz "" is a goofy cinematic...",0
1997,the kids in the hall are an acquired taste .it...,0
1998,there was a time when john carpenter was a gre...,0


### 建立Pipeline生成資料

In [3]:
# 建立Field與Dataset
text_field = data.Field(sequential=True, dtype=torch.float64, lower=True, tokenize='spacy')
label_field = data.Field(sequential=False)
input_data = data.TabularDataset(path='polarity.tsv', 
                                 format='tsv', 
                                 fields=[('text', text_field), ('label', label_field)])

In [4]:
# 取的examples並打亂順序
examples = input_data.examples
np.random.shuffle(examples)

# 以8:2的比例切分examples
split_idx = int(len(examples) * 0.8)
train_ex = examples[:split_idx]
test_ex = examples[split_idx:]

# 建立training與testing dataset
train_data = data.Dataset(examples=train_ex, fields={'text': text_field, 'label': label_field})
test_data = data.Dataset(examples=test_ex, fields={'text': text_field, 'label': label_field})

train_data[0].label, train_data[0].text

('1',
 ['wow',
  '!',
  'what',
  'a',
  'movie',
  '.it',
  "'s",
  'everything',
  'a',
  'movie',
  'can',
  'be',
  ':',
  'funny',
  ',',
  'dramatic',
  ',',
  'interesting',
  ',',
  'weird',
  ',',
  'funny',
  ',',
  'weird',
  'and',
  'strikingly',
  'original',
  '.yep',
  'that',
  'pretty',
  'much',
  'describes',
  'this',
  'movie',
  '.it',
  'starts',
  'out',
  'like',
  'a',
  'regular',
  'movie',
  'and',
  'ends',
  'up',
  'being',
  'one',
  'of',
  'the',
  'weirdest',
  ',',
  'funniest',
  'most',
  'original',
  'movies',
  'i',
  'have',
  'ever',
  'seen',
  '.it',
  'boggles',
  'the',
  'mind',
  'and',
  'some',
  'have',
  'to',
  'wonder',
  'why',
  'we',
  'can',
  'not',
  'get',
  'movies',
  'like',
  'this',
  'more',
  'often',
  '.besides',
  'being',
  'one',
  'of',
  'the',
  'best',
  'films',
  'of',
  'the',
  'year',
  ',',
  'being',
  'john',
  'malkovich',
  'may',
  'as',
  'well',
  'be',
  'one',
  'of',
  'the',
  'best',
  'mo

In [5]:
# 建立字典
text_field.build_vocab(train_data)
label_field.build_vocab(train_data)

print(f"Vocabularies of index 0-10: {text_field.vocab.itos[:10]} \n")
print(f"words to index {text_field.vocab.stoi}")

Vocabularies of index 0-10: ['<unk>', '<pad>', ',', 'the', 'a', 'and', 'of', 'to', 'is', 'in'] 

words to index defaultdict(<function _default_unk_index at 0x7f1f8629bea0>, {'<unk>': 0, '<pad>': 1, ',': 2, 'the': 3, 'a': 4, 'and': 5, 'of': 6, 'to': 7, 'is': 8, 'in': 9, "'s": 10, '"': 11, 'that': 12, '-': 13, 'it': 14, ')': 15, '(': 16, '.': 17, 'with': 18, 'as': 19, 'for': 20, 'film': 21, 'his': 22, 'this': 23, 'he': 24, '.the': 25, 'are': 26, 'on': 27, 'but': 28, 'i': 29, 'be': 30, 'by': 31, "n't": 32, 'movie': 33, 'who': 34, 'an': 35, 'not': 36, 'one': 37, 'was': 38, 'have': 39, 'from': 40, 'has': 41, 'you': 42, 'at': 43, 'her': 44, 'they': 45, 'all': 46, '?': 47, 'out': 48, 'about': 49, 'like': 50, 'up': 51, 'more': 52, 'so': 53, 'do': 54, 'or': 55, 'which': 56, ':': 57, 'their': 58, 'what': 59, '.it': 60, 'does': 61, 'some': 62, 'just': 63, 'she': 64, 'there': 65, 'him': 66, 'into': 67, 'when': 68, 'than': 69, "'": 70, 'time': 71, 'good': 72, 'only': 73, 'can': 74, 'would': 75, 'we

In [6]:
# create iterator for training and testing data
train_iter, test_iter = data.Iterator.splits(datasets=(train_data, test_data),
                                             batch_sizes=(3, 3),
                                             repeat=False,  
                                             sort_key = lambda ex: len(ex.text)) 

In [7]:
for train_batch in train_iter:
    print(train_batch.text, train_batch.text.shape)
    print(train_batch.label, train_batch.label.shape)
    break

tensor([[1.0820e+03, 9.4800e+02, 2.6630e+03],
        [8.2470e+03, 2.9780e+03, 5.7000e+01],
        [1.0000e+01, 1.0000e+01, 4.0000e+00],
        ...,
        [1.1570e+03, 1.0000e+00, 1.0000e+00],
        [2.3000e+01, 1.0000e+00, 1.0000e+00],
        [1.7000e+01, 1.0000e+00, 1.0000e+00]], dtype=torch.float64) torch.Size([587, 3])
tensor([2, 2, 1]) torch.Size([3])
